In [1]:
!pip install tensorflow

In [ ]:
import json
import tensorflow as tf
from tensorflow.keras.metrics import AUC
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras import layers, models
from tensorflow.keras.optimizers import Adam
import random
import os

from google.colab import drive

drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
!unzip /content/drive/MyDrive/Radiology/subset.zip

Streaming output truncated to the last 5000 lines.
  inflating: data/processed/Normal/masks/Normal-5764.png  
  inflating: data/processed/Normal/masks/Normal-5868.png  
  inflating: data/processed/Normal/masks/Normal-5869.png  
  inflating: data/processed/Normal/masks/Normal-587.png  
  inflating: data/processed/Normal/masks/Normal-5870.png  
  inflating: data/processed/Normal/masks/Normal-5871.png  
  inflating: data/processed/Normal/masks/Normal-5872.png  
  inflating: data/processed/Normal/masks/Normal-5873.png  
  inflating: data/processed/Normal/masks/Normal-5874.png  
  inflating: data/processed/Normal/masks/Normal-5875.png  
  inflating: data/processed/Normal/masks/Normal-5876.png  
  inflating: data/processed/Normal/masks/Normal-5877.png  
  inflating: data/processed/Normal/masks/Normal-5878.png  
  inflating: data/processed/Normal/masks/Normal-5879.png  
  inflating: data/processed/Normal/masks/Normal-588.png  
  inflating: data/processed/Normal/masks/Normal-5880.png  
  infla

## Loading data

In [4]:
import json

with open("index.json") as f:
    records = json.load(f)

print(len(records))
label_map = {
    "COVID" : 1.0,
    "Normal" : 0.0
}

random.shuffle(records)

split_idx = int(len(records) * (1 - 0.2))

train_records = records[:split_idx]
val_records = records[split_idx:]


13808


## Helper functions for train/validation dataset generator

In [5]:
def load_image(path, label):
    img = tf.io.read_file(path)
    img = tf.image.decode_png(img, channels = 3)
    img = tf.image.resize(img, (224, 224))
    img = tf.cast(img, tf.float32) / 225.0

    return img, label

In [6]:
def build_dataset(recs, training):
    paths = [os.path.join(r["image_path"]) for r in recs]
    labels = [label_map[r["label"]] for r in recs]

    ds = tf.data.Dataset.from_tensor_slices((paths, labels))
    ds = ds.map(load_image, num_parallel_calls = tf.data.AUTOTUNE)

    if training:
        ds.shuffle(512)

    ds = ds.batch(32).prefetch(tf.data.AUTOTUNE)

    return ds

## Train and Validation Datasets

In [7]:
train_ds = build_dataset(train_records, True)
val_ds = build_dataset(val_records, False)

## Load base model

In [8]:
base_model = EfficientNetB0(
    include_top = False,
    weights = "imagenet",
    input_shape = (224, 224, 3)
)
base_model.trainable = False

16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 1s 0us/step


## Top of the model

In [9]:
x = base_model.output
x = layers.GlobalAveragePooling2D()(x)
output = layers.Dense(1, activation = "sigmoid")(x)

model = models.Model(inputs = base_model.input, outputs = output)

## Model compilation

In [10]:
model.compile(
    optimizer = Adam(learning_rate = 1e-4),
    metrics = [AUC(name = "auc")],
    loss = "binary_crossentropy"
)

In [11]:
history = model.fit(
    train_ds,
    validation_data = val_ds,
    epochs = 5
)

Epoch 1/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 84s 164ms/step - auc: 0.4989 - loss: 0.6019 - val_auc: 0.5002 - val_loss: 0.5614
Epoch 2/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 23s 66ms/step - auc: 0.4941 - loss: 0.5846 - val_auc: 0.5000 - val_loss: 0.5614
Epoch 3/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 20s 59ms/step - auc: 0.4825 - loss: 0.5854 - val_auc: 0.5012 - val_loss: 0.5614
Epoch 4/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 23s 66ms/step - auc: 0.4982 - loss: 0.5843 - val_auc: 0.5000 - val_loss: 0.5614
Epoch 5/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 40s 63ms/step - auc: 0.5089 - loss: 0.5841 - val_auc: 0.5000 - val_loss: 0.5615


In [12]:
base_model.trainable = True

for layer in base_model.layers[:-40]:
    layer.trainable = False


In [13]:
model.compile(
    optimizer = Adam(learning_rate=1e-5),
    loss = "binary_crossentropy",
    metrics = [AUC(name="auc")]
)

In [14]:
history_ft = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5
)

Epoch 1/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 86s 164ms/step - auc: 0.5040 - loss: 0.6065 - val_auc: 0.5338 - val_loss: 0.5604
Epoch 2/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 24s 69ms/step - auc: 0.5083 - loss: 0.5871 - val_auc: 0.7333 - val_loss: 0.5584
Epoch 3/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 24s 69ms/step - auc: 0.5238 - loss: 0.5846 - val_auc: 0.7288 - val_loss: 0.5598
Epoch 4/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 23s 68ms/step - auc: 0.5451 - loss: 0.5821 - val_auc: 0.7626 - val_loss: 0.5566
Epoch 5/5
346/346 ━━━━━━━━━━━━━━━━━━━━ 22s 64ms/step - auc: 0.5438 - loss: 0.5824 - val_auc: 0.7827 - val_loss: 0.5557


In [15]:
model.save("efficient_net_classifier.keras")

In [16]:
from google.colab import files

files.download("efficient_net_classifier.keras")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>